### Do preprocessing

In [1]:
# import preprocess

# train_df, dev_df, dev_baseline_df, test_df, evidence_df = preprocess.return_df()
# original_train_df, original_dev_df, original_dev_baseline_df, original_test_df, original_evidence_df = preprocess.return_original_df()

In [2]:
# # print head of the dataframes
# print(train_df.head())
# print(dev_df.head())
# print(dev_baseline_df.head())
# print(test_df.head())
# print(evidence_df.head())
# print("#####################################")
# print(original_train_df.head())
# print(original_dev_df.head())
# print(original_dev_baseline_df.head())
# print(original_test_df.head())
# print(original_evidence_df.head())

### Doc2Vec encoding

In [3]:
# # Do doc2vec on the evidence_df
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from nltk.tokenize import word_tokenize
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import sent_tokenize
# import pandas as pd

# train_doc = train_df["claim_text"]
# dev_doc = dev_df["claim_text"]
# dev_baseline_doc = dev_baseline_df["claim_text"]
# test_doc = test_df["claim_text"]
# evidence_doc = evidence_df["value"]
# tagged_data_raw = (
#     # train_df["claim_text"].tolist() +
#     # dev_df["claim_text"].tolist() +
#     # dev_baseline_df["claim_text"].tolist() +
#     evidence_df["value"].tolist()
# )

# print(tagged_data_raw[0:5])

# tagged_data = [
#     TaggedDocument(words=doc, tags=[str(i)])
#     for i, doc in enumerate(tagged_data_raw)
#     if isinstance(doc, list) and all(isinstance(w, str) for w in doc)
# ]

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# model = Doc2Vec(vector_size=50, min_count=2, epochs=15, workers=8)
# model.build_vocab(tagged_data)
# model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)


# # Get vectors for claim_text
# train_df["claim_vector"] = train_df["claim_text"].apply(
#     lambda x: model.infer_vector(word_tokenize(str(x).lower())).tolist()
# )

# print("Finished vectorizing train_df claim_text")

# dev_df["claim_vector"] = dev_df["claim_text"].apply(
#     lambda x: model.infer_vector(word_tokenize(str(x).lower())).tolist()
# )

# print("Finished vectorizing dev_df claim_text")

# dev_baseline_df["claim_vector"] = dev_baseline_df["claim_text"].apply(
#     lambda x: model.infer_vector(word_tokenize(str(x).lower())).tolist()
# )

# print("Finished vectorizing dev_baseline_df claim_text")

# # Get vectors for evidence values
# evidence_df["evidence_vector"] = evidence_df["value"].apply(
#     lambda x: model.infer_vector(word_tokenize(str(x).lower())).tolist()
# )

# print("Finished vectorizing evidence_df value")

# print(train_df.head())

In [4]:
# train_df.to_json("d2v_train_df.json", orient="records", lines=True)
# dev_df.to_json("d2v_dev_df.json", orient="records", lines=True)
# dev_baseline_df.to_json("d2v_dev_baseline_df.json", orient="records", lines=True)
# test_df.to_json("d2v_test_df.json", orient="records", lines=True)
# evidence_df.to_json("d2v_evidence_df.json", orient="records", lines=True)

In [5]:
import pandas as pd

train_df = pd.read_json("d2v_train_df.json", lines=True)
dev_df = pd.read_json("d2v_dev_df.json", lines=True)
dev_baseline_df = pd.read_json("d2v_dev_baseline_df.json", lines=True)
test_df = pd.read_json("d2v_test_df.json", lines=True)
evidence_df = pd.read_json("d2v_evidence_df.json", lines=True)
print("Finished writing to json files")

print(train_df.head())

Finished writing to json files
                                          claim_text      claim_label  \
0  [scientific, evidence, co2, pollutant, higher,...         DISPUTED   
1  [el, nio, drove, record, high, global, tempera...          REFUTES   
2                 [1946, pdo, switched, cool, phase]         SUPPORTS   
3  [weather, channel, cofounder, john, coleman, p...         DISPUTED   
4  [january, 2008, capped, 12, month, period, glo...  NOT_ENOUGH_INFO   

                                           evidences  \
0  [evidence-442946, evidence-1194317, evidence-1...   
1                [evidence-338219, evidence-1127398]   
2                 [evidence-530063, evidence-984887]   
3  [evidence-1177431, evidence-782448, evidence-5...   
4  [evidence-1010750, evidence-91661, evidence-72...   

                                        claim_vector  \
0  [0.0040592896, -3.44491e-05, 0.0033489966, -0....   
1  [-0.0068075033, 0.0060098646, -0.0034204519, 0...   
2  [0.0017163944000000001

### Ranking

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

similarity_matrix = cosine_similarity(train_df["claim_vector"].tolist(), evidence_df["evidence_vector"].tolist())

In [11]:
print(similarity_matrix)

[[ 0.29946109 -0.03598205  0.23523968 ... -0.0662125   0.29998879
   0.17451646]
 [ 0.00991449  0.15712776  0.0253702  ... -0.22571504  0.07200416
   0.10200474]
 [-0.07890888 -0.06045833 -0.03968262 ... -0.03015415 -0.13361807
  -0.10356758]
 ...
 [ 0.28889166  0.06601726  0.04855557 ... -0.04772996  0.08833612
   0.18418002]
 [ 0.20228325  0.01620966  0.02580203 ...  0.26544355  0.09796254
  -0.03179346]
 [-0.05906129 -0.15176006 -0.13021627 ...  0.08945948 -0.01148854
  -0.20542444]]


In [33]:
print(np.argsort(similarity_matrix[0]))
print(-similarity_matrix[0][np.argsort(similarity_matrix[0])][-1])
print(-similarity_matrix[0][np.argsort(similarity_matrix[0])][0])

[ 753247  394333  507841 ... 1139182 1153325  233563]
-0.6140879631787988
0.630921201546683


In [32]:
print(np.argsort(similarity_matrix[:5], axis=1))

[[ 753247  394333  507841 ... 1139182 1153325  233563]
 [ 755581  730413  378438 ...   24837  346259 1128900]
 [ 672747   32632 1072587 ...  604267  845443  238382]
 [   5996  475916  282117 ... 1186160 1063520   50728]
 [ 677473  579096  179754 ...  765346  876166  238202]]


In [39]:
### Ranking 1: cosine similarity on claim_text with evidence to produce top 100 evidence for each claim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

similarity_matrix = cosine_similarity(train_df["claim_vector"].tolist(), evidence_df["evidence_vector"].tolist())

top_k = 10000
top_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]  # sort descending, get top 100 indices

# Create new column with top evidence texts for each claim
train_df["top_{top_k}_evidence"] = [
    evidence_df.iloc[indices]["key"].tolist() for indices in top_indices
]

In [44]:
total_matches = 0
n= 0 
for i in range(len(train_df)):
    k = 0
    evidence_list = train_df["evidences"].iloc[i]
    top_100_list = train_df["top_{top_k}_evidence"].iloc[i]

    for ev in evidence_list:
        if ev in top_100_list:
            k += 1
    print(f"Found {k} matches for gold standard {len(evidence_list)}")
    n += len(evidence_list)
    total_matches += k

print(f"Total matches: {total_matches}/{n}")

Found 0 matches for gold standard 3
Found 0 matches for gold standard 2
Found 0 matches for gold standard 2
Found 0 matches for gold standard 5
Found 0 matches for gold standard 5
Found 0 matches for gold standard 5
Found 0 matches for gold standard 2
Found 0 matches for gold standard 3
Found 0 matches for gold standard 5
Found 0 matches for gold standard 1
Found 0 matches for gold standard 1
Found 0 matches for gold standard 5
Found 0 matches for gold standard 5
Found 0 matches for gold standard 5
Found 0 matches for gold standard 1
Found 0 matches for gold standard 1
Found 0 matches for gold standard 3
Found 0 matches for gold standard 5
Found 0 matches for gold standard 1
Found 0 matches for gold standard 4
Found 0 matches for gold standard 2
Found 0 matches for gold standard 2
Found 0 matches for gold standard 4
Found 0 matches for gold standard 3
Found 0 matches for gold standard 4
Found 0 matches for gold standard 2
Found 0 matches for gold standard 1
Found 0 matches for gold sta